In [1]:
import time
import pathlib
import pandas as pd 
from selenium import webdriver

Make DataFrame

In [2]:
df_youtube = pd.DataFrame(columns = ['date_published', 'title', 'views', 'keywords', 'video_id'])

Define function for scraping the following data from Youtube: date, title, views and ID

In [3]:
def youtube_channel(extension, count, df=df_youtube): 
    '''
    Transforming df_youtube DataFrame:
    1) Initialize and run chromedriver;
    2) Find the chanel, open videos tab and scrape date, title, views and ID of "count"-number latest videos;
    3) Assign each data category to each column in DataFrame.
    '''
    
    driver = webdriver.Chrome('chromedriver\\chromedriver.exe')
    youtube = 'https://www.youtube.com/'
    driver.get(f'{youtube}results?search_query={extension}')
    time.sleep(1.5)
    channel_entrance = driver.find_element_by_css_selector('#main-link')
    channel_entrance.click()
    time.sleep(1.5)
    channel_videos_entrance = driver.find_element_by_css_selector('#tabsContent > tp-yt-paper-tab:nth-child(4) > div')
    channel_videos_entrance.click()
    time.sleep(2.5)
    part_one = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['
    part_two = ']/div[1]/div[1]/div[1]/h3/a'
    date_list, title_list, views_list, id_list  = [], [] , [] ,[]
    
    for r in range(1, count + 1):
        url = part_one + str(r) + part_two
        videos_entrance = driver.find_element_by_xpath(url)
        driver.execute_script("arguments[0].click();", videos_entrance)
        time.sleep(1.5)
        date_list.append(driver.find_element_by_css_selector('#date > yt-formatted-string').text)
        title_list.append(driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text)
        views_list.append(driver.find_element_by_css_selector('#count > ytd-video-view-count-renderer > span.view-count.style-scope.ytd-video-view-count-renderer').text)
        time.sleep(2.5)
        video_ids = driver.find_elements_by_xpath('//*[@id="page-manager"]/ytd-watch-flexy')
        
        for video in video_ids:
            video_id = str(video.get_attribute('video-id'))
            id_list.append(video_id)
        driver.back()
        
    df['date_published'] = date_list
    df['title'] = title_list
    df['views'] = views_list
    df['video_id'] = id_list
    driver.quit()

Define function to scrape keywords

In [4]:
def youtube_keywords(df=df_youtube, overall_list = [] ):
    '''
    Transforming df_youtube DataFrame:
    1) Initialize and run chromedriver;
    2) Scrape tags through each video from DataFrame's video_id column;
    3) Assign each list of tags to coresponding column in DataFrame.
    '''
    
    for video_id in list(df['video_id']):
        tag_list = [] 
        driver = webdriver.Chrome('chromedriver\\chromedriver.exe')
        youtube = 'https://www.youtube.com/'
        driver.get(f'{youtube}watch?v={video_id}')
        time.sleep(2)
        tags = driver.find_elements_by_xpath("//meta[@property='og:video:tag']") 
        
        for tag in tags:
            video_tag = str(tag.get_attribute('content'))     
            tag_list.append(video_tag)  
        overall_list.append(tag_list)   
        driver.quit() 
    
    for i in range(len(df['keywords'])):
        df['keywords'][i] = overall_list[i]

Define function to convert to proper types date, view and keywords

In [5]:
def df_youtube_typization(df=df_youtube):
    '''
    Transforming df_youtube DataFrame:
    1) Convert keywords column type to tuple;
    2) Convert views column type to integer;
    3) Convert date_published column type to date.
    '''
    
    for i in range(len(df['keywords'])):
        if len(df['keywords'][i]) == 0:
            continue
        else:
            df['keywords'][i] = repr(tuple(df['keywords'][i]))
        
    for i in range(len(df['views'])):
        if ' views' in df['views'][i]:
            result = df['views'][i].find(' views')
            df['views'][i] = int(df['views'][i][:result].replace(',',''))
        elif 'watching' in df['views'][i]:
            df['views'][i] = 'still online'
            
    df['date_published'] = pd.to_datetime(df['date_published'], errors='ignore')

Run **youtube_channel()** function to extract date, title, views and ID

In [6]:
youtube_channel(input('Enter your channel: ').replace(' ','+'), int(input('Enter amount of latest videos: ')))
df_youtube

Enter your channel: skirmantas malinauskas
Enter amount of latest videos: 8


,date_published,title,views,keywords,video_id
0,Started streaming 57 minutes ago,Atsakymas Šviniui,"6,119 watching now",NaN,MFzRWROZthM
1,14 Mar 2021,Henrikas Daktaras: Mes su Šviniumi buvome vers...,"144,845 views",NaN,gQayDUCH05Q
2,7 Mar 2021,Kaip pavogti milijoną?,"78,144 views",NaN,AdUiTzUNrd8
3,28 Feb 2021,Apie tai kalbėti draudžiama,"86,302 views",NaN,Aq-bpR6vFig
4,16 Feb 2021,"""CityBee"" mums pamelavo - 110 tūkstančių žmoni...","148,019 views",NaN,wbKznbxe5r4
5,14 Feb 2021,Sudužusios svajonės,"103,251 views",NaN,DyXAs7fDyus
6,24 Jan 2021,Palikta viena prieš šiukšlių mafiją,"81,911 views",NaN,DyZ3v0DRPGw
7,Streamed live on 21 Jan 2021,Geras vakaras pokalbiui :),"90,370 views",NaN,DlnsrZDpHx0


Run **youtube_keywords()** function to extract tags directly to *df_youtube* DataFrame

In [7]:
youtube_keywords()
df_youtube

,date_published,title,views,keywords,video_id
0,Started streaming 57 minutes ago,Atsakymas Šviniui,"6,119 watching now",[],MFzRWROZthM
1,14 Mar 2021,Henrikas Daktaras: Mes su Šviniumi buvome vers...,"144,845 views","[henrikas daktaras, henrikas, pukelis, arūnas ...",gQayDUCH05Q
2,7 Mar 2021,Kaip pavogti milijoną?,"78,144 views",[],AdUiTzUNrd8
3,28 Feb 2021,Apie tai kalbėti draudžiama,"86,302 views","[Algirdas Toliatas, Paulius Vaineikis, Valdas ...",Aq-bpR6vFig
4,16 Feb 2021,"""CityBee"" mums pamelavo - 110 tūkstančių žmoni...","148,019 views","[CityBee, asmens duomenys, vagystė, automobili...",wbKznbxe5r4
5,14 Feb 2021,Sudužusios svajonės,"103,251 views","[žalgiris, Vygandas Jankūnas, KlasJet, gedimin...",DyXAs7fDyus
6,24 Jan 2021,Palikta viena prieš šiukšlių mafiją,"81,911 views","[simonas gentvilas, gentvilas, marijus gailius...",DyZ3v0DRPGw
7,Streamed live on 21 Jan 2021,Geras vakaras pokalbiui :),"90,370 views","[Pukelis, Arunas Pukleis, Svinius, Švinius, ve...",DlnsrZDpHx0


Run **df_youtube_typization()** function to convert *date_published* to date type (if it possible), *views* to integer and *keywords* to tuple

In [8]:
df_youtube_typization()
df_youtube

,date_published,title,views,keywords,video_id
0,Started streaming 57 minutes ago,Atsakymas Šviniui,still online,[],MFzRWROZthM
1,14 Mar 2021,Henrikas Daktaras: Mes su Šviniumi buvome vers...,144845,"('henrikas daktaras', 'henrikas', 'pukelis', '...",gQayDUCH05Q
2,7 Mar 2021,Kaip pavogti milijoną?,78144,[],AdUiTzUNrd8
3,28 Feb 2021,Apie tai kalbėti draudžiama,86302,"('Algirdas Toliatas', 'Paulius Vaineikis', 'Va...",Aq-bpR6vFig
4,16 Feb 2021,"""CityBee"" mums pamelavo - 110 tūkstančių žmoni...",148019,"('CityBee', 'asmens duomenys', 'vagystė', 'aut...",wbKznbxe5r4
5,14 Feb 2021,Sudužusios svajonės,103251,"('žalgiris', 'Vygandas Jankūnas', 'KlasJet', '...",DyXAs7fDyus
6,24 Jan 2021,Palikta viena prieš šiukšlių mafiją,81911,"('simonas gentvilas', 'gentvilas', 'marijus ga...",DyZ3v0DRPGw
7,Streamed live on 21 Jan 2021,Geras vakaras pokalbiui :),90370,"('Pukelis', 'Arunas Pukleis', 'Svinius', 'Švin...",DlnsrZDpHx0
